In [ ]:
import os
import time
import numpy as np
import pynq
from pynq import Overlay, allocate
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

# ===================================================================
#                       配置信息
# ===================================================================
BITSTREAM_FILE = 'sys4.bit' 
INPUT_FILE = 'images_2_volume.npy'  # 修改为您的文件名
OUTPUT_FILE = 'processed_data.npy'  # 输出文件名
OUTPUT_PNG_FOLDER = 'image_2'  # PNG序列输出文件夹
PROCESSING_WIDTH = 512
PROCESSING_HEIGHT = 512
NUM_FRAMES = 108  # 添加帧数信息

# --- 从 preprocess_dataset.py 提取的归一化常量 ---
MIN_CLIP_VAL = 0.0
MAX_CLIP_VAL = 350.0

# ===================================================================
#             PNG输出模块
# ===================================================================
def save_frames_as_png_sequence(frames, output_folder, prefix="i", start_index=0):
    """
    将处理后的帧保存为PNG序列
    - frames: numpy数组，形状为(num_frames, height, width)或(num_frames, height, width, 3)
    - output_folder: 输出文件夹路径
    - prefix: 文件名前缀
    - start_index: 起始索引
    """
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    
    print(f"正在保存PNG序列到文件夹: {output_folder}")
    
    saved_count = 0
    for frame_idx in tqdm(range(frames.shape[0]), desc="保存PNG序列"):
        # 获取当前帧
        if len(frames.shape) == 4:  # (height, width, 3)
            frame = frames[frame_idx]
        else:  # (height, width), 转换为RGB
            frame = np.stack((frames[frame_idx],) * 3, axis=-1)
        
        # 确保数据类型为uint8
        if frame.dtype != np.uint8:
            frame = frame.astype(np.uint8)
        
        # 生成文件名: i0xxx,0000b.png 格式
        frame_number = start_index + frame_idx
        filename = f"{prefix}{frame_number:04d},0000b.png"
        filepath = os.path.join(output_folder, filename)
        
        # 使用PIL保存PNG图像
        try:
            # 如果是RGB图像，直接保存
            if frame.shape[-1] == 3:
                img = Image.fromarray(frame, 'RGB')
            else:
                img = Image.fromarray(frame, 'L')
            
            img.save(filepath, 'PNG')
            saved_count += 1
        except Exception as e:
            print(f"保存第{frame_idx}帧时出错: {e}")
    
    print(f"成功保存 {saved_count} 张PNG图像到 {output_folder}")
    return saved_count

# ===================================================================
#             核心NPY处理函数 (采用新归一化标准)
# ===================================================================
def process_multiframe_npy_with_fpga(fpga_ip, dma, input_path, output_path, input_buffer, output_buffer):
    """
    使用FPGA处理多帧.npy文件。
    - 加载(108,512,512,3)格式的npy文件（RGB灰度CT图像）
    - 转换为(108,512,512)灰度格式进行处理
    - 对每一帧按照指定标准进行裁剪和归一化，并转换为uint8 RGB格式
    - 通过DMA将数据发送到FPGA IP核进行处理
    - 将处理结果存为新的.npy文件
    """
    try:
        # 1. 加载数据
        float_array = np.load(input_path)
        print(f"输入数据形状: {float_array.shape}")
        
        # 验证输入形状 - 支持(108,512,512,3)格式
        if float_array.shape == (NUM_FRAMES, PROCESSING_HEIGHT, PROCESSING_WIDTH, 3):
            print("检测到RGB格式输入，转换为灰度格式...")
            # 将RGB转换为灰度（使用标准加权平均）
            gray_array = np.dot(float_array[...,:3], [0.299, 0.587, 0.114])
            print(f"转换后灰度数据形状: {gray_array.shape}")
        elif float_array.shape == (NUM_FRAMES, PROCESSING_HEIGHT, PROCESSING_WIDTH):
            print("检测到灰度格式输入，直接处理...")
            gray_array = float_array
        else:
            print(f"错误: 输入数据形状 {float_array.shape} 与预期形状不匹配")
            print(f"预期形状: {(NUM_FRAMES, PROCESSING_HEIGHT, PROCESSING_WIDTH)} 或 {(NUM_FRAMES, PROCESSING_HEIGHT, PROCESSING_WIDTH, 3)}")
            return False
        
        # 确保数据类型为float64进行处理
        if gray_array.dtype != np.float64:
            gray_array = gray_array.astype(np.float64)
        
        # 初始化输出数组
        processed_frames = np.empty((NUM_FRAMES, PROCESSING_HEIGHT, PROCESSING_WIDTH, 3), dtype=np.uint8)
        
        # 处理每一帧
        for frame_idx in tqdm(range(NUM_FRAMES), desc="处理帧"):
            # 2. 应用新的归一化标准
            #    第一步: 按照 preprocess_dataset.py 的标准裁剪数值范围
            clipped_array = np.clip(gray_array[frame_idx], MIN_CLIP_VAL, MAX_CLIP_VAL)
            
            #    第二步: 将裁剪后的范围 [-79.0, 304.0] 线性缩放到 [0.0, 1.0]
            if MAX_CLIP_VAL - MIN_CLIP_VAL > 0:
                normalized_array = (clipped_array - MIN_CLIP_VAL) / (MAX_CLIP_VAL - MIN_CLIP_VAL)
            else:
                normalized_array = np.zeros(clipped_array.shape, dtype=np.float64)

            #    第三步: 缩放到 [0, 255] 并转换为硬件所需的 uint8 类型
            uint8_array = (normalized_array * 255.0).astype(np.uint8)

            # 3. 单通道灰度图堆叠为三通道RGB图以匹配DMA缓冲区
            img_np_8bit_rgb = np.stack((uint8_array,) * 3, axis=-1)

            # 4. 将处理好的uint8数据复制到DMA输入缓冲区
            np.copyto(input_buffer, img_np_8bit_rgb)

            # 5. 配置并启动 Filter2D IP 核
            fpga_ip.write(0x10, PROCESSING_HEIGHT) # 设置图像高度
            fpga_ip.write(0x18, PROCESSING_WIDTH)  # 设置图像宽度
            fpga_ip.write(0x00, 1)                 # 启动IP核

            # 6. 执行DMA传输
            dma.sendchannel.transfer(input_buffer)
            dma.recvchannel.transfer(output_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.wait()
            
            # 7. 保存处理结果
            processed_frames[frame_idx] = output_buffer.copy()
        
        # 8. 保存所有处理后的帧
        np.save(output_path, processed_frames)
        
        # 9. 保存PNG序列
        save_frames_as_png_sequence(processed_frames, OUTPUT_PNG_FOLDER, prefix="i", start_index=0)
        
        return True
        
    except Exception as e:
        print(f"  [错误] 处理 {input_path} 时发生错误: {e}")
        import traceback
        traceback.print_exc()
        return False

# ===================================================================
#                       主程序入口
# ===================================================================
def main():
    print("PYNQ .npy 多帧文件处理程序 (适配 Filter2D IP - 新归一化标准 + PNG输出)")
    print("="*60)

    if not os.path.exists(INPUT_FILE):
        print(f"[错误] 输入文件 '{INPUT_FILE}' 不存在。")
        return
        
    # --- 硬件初始化 ---
    print(f"正在加载Overlay: {BITSTREAM_FILE}...")
    overlay = Overlay(BITSTREAM_FILE)
    print("Overlay加载成功！")
    
    dma = overlay.axi_dma_0
    filter_2d_ip = overlay.Filter2D_accel_0 
    print("成功获取 DMA 和 Filter2D IP 控制器。")
    
    # --- 内存预分配 ---
    input_buffer = None
    output_buffer = None
    try:
        print(f"正在为尺寸 {PROCESSING_WIDTH}x{PROCESSING_HEIGHT} 预分配DMA内存...")
        shape = (PROCESSING_HEIGHT, PROCESSING_WIDTH, 3)
        input_buffer = allocate(shape=shape, dtype=np.uint8)
        output_buffer = allocate(shape=shape, dtype=np.uint8)
        print("内存预分配成功！")
        
        # --- 处理文件 ---
        print(f"\n开始处理文件: {INPUT_FILE}")
        
        start_time_total = time.time()
        
        success = process_multiframe_npy_with_fpga(
            filter_2d_ip, dma, INPUT_FILE, OUTPUT_FILE, input_buffer, output_buffer
        )
        
        # --- 结果汇总 ---
        end_time_total = time.time()
        print("\n" + "="*60)
        if success:
            print("处理完成！")
            print(f"总耗时: {end_time_total - start_time_total:.4f} 秒")
            print(f"处理结果已保存至 '{OUTPUT_FILE}'")
            print(f"PNG序列已保存至 '{OUTPUT_PNG_FOLDER}' 文件夹")
        else:
            print("处理失败！")

    finally:
        # --- 统一释放内存 ---
        print("\n正在释放DMA内存...")
        if input_buffer is not None:
            input_buffer.close()
        if output_buffer is not None:
            output_buffer.close()
        print("内存已释放。程序结束。")

if __name__ == "__main__":
    main()

In [ ]:
# ===================================================================
#                       测试和验证模块
# ===================================================================
import numpy as np
import os
from PIL import Image

def test_input_format_compatibility():
    """
    测试输入格式兼容性
    """
    print("=" * 50)
    print("测试输入格式兼容性")
    print("=" * 50)
    
    # 检查输入文件
    input_file = 'images_2_volume.npy'
    if not os.path.exists(input_file):
        print(f"错误: 输入文件 '{input_file}' 不存在")
        return False
    
    try:
        # 加载并检查输入数据
        data = np.load(input_file)
        print(f"输入数据形状: {data.shape}")
        print(f"输入数据类型: {data.dtype}")
        print(f"输入数据范围: [{data.min():.2f}, {data.max():.2f}]")
        
        # 测试形状转换
        if data.shape == (108, 512, 512, 3):
            print("✓ 检测到RGB格式，测试灰度转换...")
            gray_data = np.dot(data[...,:3], [0.299, 0.587, 0.114])
            print(f"转换后灰度数据形状: {gray_data.shape}")
            print(f"灰度数据范围: [{gray_data.min():.2f}, {gray_data.max():.2f}]")
            print("✓ RGB到灰度转换成功")
        elif data.shape == (108, 512, 512):
            print("✓ 检测到灰度格式，无需转换")
        else:
            print(f"✗ 不支持的输入格式: {data.shape}")
            return False
        
        # 测试归一化
        print("✓ 测试归一化处理...")
        test_slice = gray_data[0] if data.shape == (108, 512, 512, 3) else data[0]
        
        # 应用裁剪
        clipped = np.clip(test_slice, -79.0, 304.0)
        print(f"裁剪后范围: [{clipped.min():.2f}, {clipped.max():.2f}]")
        
        # 应用归一化
        normalized = (clipped - (-79.0)) / (304.0 - (-79.0))
        print(f"归一化后范围: [{normalized.min():.4f}, {normalized.max():.4f}]")
        
        # 转换为uint8
        uint8_data = (normalized * 255.0).astype(np.uint8)
        print(f"uint8转换后范围: [{uint8_data.min()}, {uint8_data.max()}]")
        
        print("✓ 归一化处理测试成功")
        
        return True
        
    except Exception as e:
        print(f"✗ 测试过程中出错: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_png_output_functionality():
    """
    测试PNG输出功能
    """
    print("\n" + "=" * 50)
    print("测试PNG输出功能")
    print("=" * 50)
    
    try:
        # 创建测试数据
        test_frames = np.random.randint(0, 256, (5, 512, 512, 3), dtype=np.uint8)
        test_folder = 'test_png_output'
        
        print(f"创建测试数据: {test_frames.shape}")
        
        # 测试PNG保存
        from tqdm import tqdm
        
        # 创建输出文件夹
        os.makedirs(test_folder, exist_ok=True)
        
        saved_count = 0
        for frame_idx in tqdm(range(test_frames.shape[0]), desc="测试保存PNG"):
            frame = test_frames[frame_idx]
            frame_number = frame_idx
            filename = f"i{frame_number:04d}b.png"
            filepath = os.path.join(test_folder, filename)
            
            img = Image.fromarray(frame, 'RGB')
            img.save(filepath, 'PNG')
            saved_count += 1
        
        print(f"✓ 成功保存 {saved_count} 张测试PNG图像")
        
        # 验证文件存在
        saved_files = os.listdir(test_folder)
        print(f"✓ 输出文件夹包含 {len(saved_files)} 个文件")
        
        # 清理测试文件
        import shutil
        shutil.rmtree(test_folder)
        print("✓ 测试文件清理完成")
        
        return True
        
    except Exception as e:
        print(f"✗ PNG输出测试失败: {e}")
        import traceback
        traceback.print_exc()
        return False

def check_dependencies():
    """
    检查必要的依赖包
    """
    print("\n" + "=" * 50)
    print("检查依赖包")
    print("=" * 50)
    
    required_packages = {
        'numpy': 'numpy',
        'PIL': 'Pillow',
        'tqdm': 'tqdm', 
        'pynq': 'pynq',
        'matplotlib': 'matplotlib'
    }
    
    missing_packages = []
    
    for package_name, import_name in required_packages.items():
        try:
            __import__(import_name)
            print(f"✓ {package_name} 已安装")
        except ImportError:
            print(f"✗ {package_name} 未安装")
            missing_packages.append(package_name)
    
    if missing_packages:
        print(f"\n缺少依赖包: {', '.join(missing_packages)}")
        print("请使用以下命令安装:")
        print(f"pip install {' '.join(missing_packages)}")
        return False
    else:
        print("\n✓ 所有依赖包已正确安装")
        return True

# 执行所有测试
def run_all_tests():
    """
    运行所有测试
    """
    print("开始运行预处理器测试套件...")
    
    tests = [
        ("依赖包检查", check_dependencies),
        ("输入格式兼容性", test_input_format_compatibility), 
        ("PNG输出功能", test_png_output_functionality)
    ]
    
    results = {}
    for test_name, test_func in tests:
        print(f"\n正在运行测试: {test_name}")
        results[test_name] = test_func()
    
    # 汇总结果
    print("\n" + "=" * 50)
    print("测试结果汇总")
    print("=" * 50)
    
    all_passed = True
    for test_name, result in results.items():
        status = "✓ 通过" if result else "✗ 失败"
        print(f"{test_name}: {status}")
        if not result:
            all_passed = False
    
    if all_passed:
        print("\n🎉 所有测试通过！预处理器已准备就绪。")
    else:
        print("\n⚠️  部分测试失败，请检查相关配置。")
    
    return all_passed

# 运行测试
if __name__ == "__main__" or '__file__' in globals():
    run_all_tests()